In [2]:
import cv2
import os

# processing the video data

In [3]:
import cv2
import os

def extract_frames(video_path, output_folder):
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imwrite(os.path.join(output_folder, f"frame_{count}.jpg"), frame)
        count += 1
    cap.release()


extract_frames("acc_video.mp4", r"C:\Users\PINKY\OneDrive\Desktop\acc_det\videoplayback (online-video-cutter.com).mp4\acc_video.mp4")


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator             #LOAD IMAGE FROM TRAIN N TEST FOLDERS

train_dir = r"C:\Users\PINKY\OneDrive\Desktop\acc_det\datasets\train"
test_dir = r"C:\Users\PINKY\OneDrive\Desktop\acc_det\datasets\test"

datagen = ImageDataGenerator(rescale=1./255)    #Normalize pixel values (0-1)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=16,
    class_mode='binary'   #Binary classification (accident vs non-accident)
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=16,
    class_mode='binary'
)


Found 791 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential                          # Sequential is linear stack of layers, used to define the CNN model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([                                                                                # Build the Model
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')                                                                  # 1 = Accident, 0 = No Accident
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train and Save the Model using the Generator
model.fit(train_generator, epochs=10, validation_data=test_generator)
model.save("accident_detection_model.h5",r"C:\Users\PINKY\OneDrive\Desktop\acc_det\model")


C:\Users\PINKY\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\PINKY\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 27s 506ms/step - accuracy: 0.5217 - loss: 0.8446 - val_accuracy: 0.5700 - val_loss: 0.6810
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 157ms/step - accuracy: 0.6146 - loss: 0.6520 - val_accuracy: 0.5200 - val_loss: 0.6491
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 163ms/step - accuracy: 0.6767 - loss: 0.5924 - val_accuracy: 0.6400 - val_loss: 0.6315
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 159ms/step - accuracy: 0.7133 - loss: 0.5705 - val_accuracy: 0.6900 - val_loss: 0.5825
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 163ms/step - accuracy: 0.7679 - loss: 0.4991 - val_accuracy: 0.7600 - val_loss: 0.4704
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 160ms/step - accuracy: 0.7876 - loss: 0.4389 - val_accuracy: 0.8100 - val_loss: 0.4051
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 163ms/step - accuracy: 0.8812 - loss: 0.3149 - val_accuracy: 0.8700 - val_loss: 0.3264
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 158ms/step - accuracy: 0.8966 - loss: 0.2563 - val_accuracy: 0

# implementation of accident detection in real time

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import os

model_path=r"C:\Users\PINKY\OneDrive\Desktop\acc_det\accident_detection_model.h5"
model = load_model(model_path)

def detection_acc():
    cap = cv2.VideoCapture(0)

    frame_count=0
    accident_frames=0 

    text ="Processing...."
    color= (255,255,255)
    

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, (64, 64))                                  # Resize & Normalize
        reshaped_frame = np.expand_dims(resized_frame, axis=0) / 255.0  

        prediction = model.predict(reshaped_frame)[0][0]                         # Predict Accident
        print(f"Prediction score: {prediction}")

        frame_count+=1
        if prediction > 0.7:                                                       # Prediction
            accident_frames+=1
            
        if frame_count >=10:
            if accident_frames>=5:
                text = "ACCIDENT DETECTED"
                color = (0, 0, 255)     #red
            else:
                text = "No Accident detected"
                color = (0, 255, 0)  #green

            frame_count=0
            accident_frames=0

        cv2.putText(frame, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.imshow("Accident Detection", frame)

        if cv2.waitKey(25) & 0xFF == ord('x'):                      # Press 'x' to exit
            break

    cap.release()
    cv2.destroyAllWindows()

detection_acc()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
Prediction score: 0.36765703558921814
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Prediction score: 0.4412311315536499
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Prediction score: 0.23440198600292206
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Prediction score: 0.5417748689651489
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Prediction score: 0.3450557589530945
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Prediction score: 0.27101075649261475
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Prediction score: 0.24266937375068665
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Prediction score: 0.27026525139808655
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Prediction score: 0.27755895256996155
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Prediction score: 0.27488893270492554
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Prediction score: 0.27313798666000366
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Prediction score: 0.253255158662796
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Prediction score: 0.2784139811992645
1/1 ━━━━━━━━━━━━━